In [1]:
# %pip install -q -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import os
import ast
from dotenv import load_dotenv
load_dotenv('test.env')  # take environment variables from .env.

import pandas as pd
import numpy as np
from openai import AzureOpenAI
# Import date class from datetime module
from datetime import date


from langchain_openai import ChatOpenAI
from langchain import LLMChain
from dotenv import load_dotenv

## TEST LANG CHAIN

In [4]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [4]:
sample_image_url1 = 'https://kimlygroup.sg/wp-content/uploads/2020/12/halo_29662-1.jpg'
sample_image_url2 = 'https://kimlygroup.sg/wp-content/uploads/2020/12/D1-2-pax-Meal-Cereal-dory-fish-Broccoli-Fried-Eggs.png'
sample_image_url3 = 'https://live.staticflickr.com/65535/48346712292_41bba7d636_b.jpg'
sample_image_url4 = 'https://cdn.copymethat.com/media/how-to-make-singapore-hainanese-roasted--2017020412515082466844wck.jpg'
prata_image = 'https://cache.travelfish.org/blogs/singapore/wp-content/uploads/2012/02/roti_prata.jpg'
mutton_briyani = 'https://www.halaltag.com/images/photo/3285-photo.jpg'


In [5]:
#create the model for the Vision 
# client = AzureOpenAI(
#     api_version=os.getenv("OPENAI_API_VERSION"),
#     azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
#     api_key=os.getenv("OPENAI_API_KEY"),
# )
openai_api_key = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(model=os.getenv("GPT_MODEL"))

In [2]:
with open('prompt_and_examples/system_prompt.txt', 'r') as file:
    system_message = file.read()

print(system_message)

Task: You are an AI assistant specializing in nutritional analysis. Your task is to analyze dishes and provide detailed macronutrient and calorie information based on specific criteria. Always follow these instructions:

Output Format: Provide all results in JSON format only.

Transformation Points:
1. Dish Analysis:
   - Identify the dish name and weight from the provided tuple.
   - Accurately deduce the amount of each macronutrient (carbohydrates, protein, fats) contained within the dish.
   - Calculate the total calories based on the macronutrient composition.

2. Macronutrient Calculation:
   - Use the following calorie conversion:
     * 1 gram of carbohydrate = 4 kcal
     * 1 gram of protein = 4 kcal
     * 1 gram of fat = 9 kcal
   - Ensure that the sum of calories from all macronutrients equals the total calories reported.

3. Output Formatting:
   - Present the results in a dictionary format with specific key-value pairs.
   - Ensure the output string can be converted into v

In [ ]:
system_message_indiv = """
         You are an expert in the field of nutrition and a dietician.
         You are able to accurately deduce the amount of each macronutrients (carbohydrates, protein, fats) contained within a dish if you are given the weight of the dish in grams.
         You know that the calorie for 1 gram of carbohydrate = 4kcal, 1 gram protein = 4kcal, and 1 gram of fat = 9kcal.
         As such, you would then use the combination of macroproteins to calculate the total amount of calories within the dish to ensure that you succeed at your job and get paid in $100 in tips.
            
         
         Instructions:
         - Given a tuple with 2 elements: (Name of the dish, and weight in grams). You are able to identify the calories, and macronutrient content in grams
         - You would then output the results as a dictionary with a key-value of :
             - KEY = dish, VALUE = the name of the dish
             - KEY = weight, VALUE = the weight of the dish
             - KEY = carbohydrate, VALUE = the amount of carbohydrates within the provided meal, in units of grams
             - KEY = protein, VALUE = the amount of protein within the provided meal, in units of grams
             - KEY = fats, VALUE = the amount of fats within the provided meal, in units of grams
             - KEY = calories, VALUE = the amount of calories the dish contains with respect to the combination of macronutrients.
         - You would also ensure that the kcal combination of the three macronutrient will accurately sum up to the total calories identified.
         - Only output the result in the provided format and remove all other commentaries. Ensure that the string output is able to be converted into a json.
         - The output's format would look like this:

         "{{ 'dish': 'Hainanese chicken rice [Rice only]',
             'weight': '200',
             'carbohydrate': '80',
             'protein' = '10',
             'fats' = '10',
            'total calories': '450'
             }}"

        The above output is valid because:
        - 80 grams of carbohydrate = 80 grams * 4 kcal = 320 kcal
        - 10 grams of protein = 10 grams * 4kcal = 40 kcal
        - 10 grams of fats = 10 grams * 9 kcal = 90 kcal
        - Summing the values up, 320kcal from carbohydrate + 40 kcal from protein + 90 kcal from fats = 450 kcal total, which is the total calories stated in the output
        
        Possible Output Examples:

        Example 1:
         "{{ 'dish': 'Roasted Chicken Breast',
             'weight': '100',
             'carbohydrate': '0',
             'protein' = '40',
             'fats' = '6',
             'total calories': '214'
             }}"

        Example 1 explanation: 
        The above output is valid and you get paid $100 in tips because
            - 0 grams of carbohydrate = 0 grams * 4 kcal = 0 kcal
            - 40 grams of protein = 40 grams * 4kcal = 160 kcal
            - 6 grams of fats = 6 grams * 9 kcal = 54 kcal
            - Summing the values up, 0 Kcal from carbohydrate + 160 kcal from protein + 54 kcal from fats = 214 kcal total, which is the total calories stated in the output

        Example 2:
         "{{ 'dish': 'mixed vegetable',
             'weight': '80',
             'carbohydrate': '10',
             'protein' = '2',
             'fats' = '0',
             'total calories': '50'
             }}"

        Example 2 explanation: 
        The above output is not valid because
            - 10 grams of carbohydrate = 10 grams * 4 kcal = 40 kcal
            - 2 grams of protein = 2 grams * 4kcal = 8 kcal
            - 0 grams of fats = 0 grams * 9 kcal = 0 kcal
            - Summing the values up,  40 Kcal from carbohydrate + 8 kcal from protein + 0 kcal from fats = 48 kcal total.
            - This is different from the stated total calories which 50 kcal.
            - As such, your answer is wrong and you don't get paid $100 in tips.
         """

system_message_indiv_prompt = SystemMessagePromptTemplate.from_template(system_message_indiv) 

In [11]:
# Create a variable called {user_input} that is used to populate the prompt
human_template="{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template) 
chat_prompt = ChatPromptTemplate.from_messages([system_message_indiv_prompt, human_message_prompt]) 

chain = LLMChain(llm=llm, prompt=chat_prompt)

## START TO END TEST

In [15]:
def analyze_picture(image_url):
    #uses the vision model
    picture_gpt = client.chat.completions.create(
        model='gpt-4-turbo-vision',
        seed = 888,
        messages=[
            { "role": "system",
             "content": """You are an expert in the field of nutrition and a dietician. You are based in Singapore.
             As such, our clients are usually Asian and have meal consisting of mostly oriental dishes that originates from Singapore.
             
             Instructions:
             - Given a picture of a meal, you are able to identify the different dishes seen in the picture and estimate the portion sizing.
             - You would then output the results as a dictionary with a key-value of :
                 - KEY = dishes, VALUE = a list of tuples, with each tuple containing 2 elements. The first element being the dish identified, the second element being the estimated weight of the dish in grams with respect to the picture provided
                 - KEY = recommendation, VALUE =  a possible recommendation on how to improve their meal choices.
             - Only output the result in the provided format and remove all other commentaries. Ensure that the string output is able to be converted into a json.
             - The output's format would look like this:
    
             "{ 'dishes':
                 [('Hainanese chicken rice [Rice only]', '200'), ('roasted chicken breast', '150'), ('mixed vegetables', '100'), ('steamed egg', '120')],
                 'recommendation': 'This looks like a balanced diet for one individual. Great choices!'
                 }"
    
            Possible Output Examples:
    
            Example 1:
            "{ 'dishes':
                 [('white rice', '200'), ('Stir Fry French beans', '80'), ('Stir Fry Scrambled Eggs', '100'), ('Fried Chicken Wing', '150')],
                 'This looks like balanced diet for one individual but most things are oily and fried.'
                 }"
    
            Example 2:
            "{ 'dishes':
                 [('white rice', '200'), ('white rice', '200'),  ('sweet and sour pork', '300'), ('Sambal Kang Kong', '200'), ('Fried Chicken Cutlet', '200')],
                 'This looks like a meal for two individuals. It looks like it is heavy in protein and would be better balanced with more vegetables!'
                 }"
             """
            },
            { "role": "user",
             "content": [  
                { 
                    "type": "text", 
                    "text": "What is in my meal?" 
                },
                { 
                    "type": "image_url",
                    "image_url": {
                        "url": image_url
                    }
                }
            ] } 
        ]
        #[!IMP] - Max tokens parameter is required, otherwise the API cuts off the response randomly
        ,max_tokens=4000
        , temperature = 0.0
    )

    return ast.literal_eval(str(picture_gpt.choices[0].message.content)[1:-1])


In [27]:
def summarize_calories(image_url):

    picture_analysis = analyze_picture(image_url)
    dish_list = picture_analysis['dishes']
    recommendation = picture_analysis['recommendation']

    result = { 'dish_list': [], 'total_kcal': 0, 'total_carbohydrates': 0, 'total_protein': 0, 'total_fats': 0, 'recommendation': recommendation }

    for i in dish_list:
        dish_result = (ast.literal_eval(str(chain.run(user_input= i))[1:-1]))
        result['dish_list'].append(dish_result)
        result['total_kcal'] += int(dish_result['total calories'])
        result['total_carbohydrates'] += int(dish_result['carbohydrate'])
        result['total_protein'] += int(dish_result['protein'])
        result['total_fats'] += int(dish_result['fats'])

    ingredients = "\n".join([f"- {dish['dish']}, weight: {dish['weight']}g, {dish['carbohydrate']}g carbs, {dish['protein']}g protein, {dish['fats']}g fats, {dish['total calories']}kcal" for dish in result['dish_list']])
    
    formatted_string = f"""
Ingredients include:
{ingredients}

Total Calories: {result['total_kcal']}kcal
Total Carbohydrates = {result['total_carbohydrates']}g
Total Protein = {result['total_protein']}g
Total Fats = {result['total_fats']}g

Recommendation for this meal choice:
{result['recommendation']}"""

    print(formatted_string)
    return
        

In [28]:
summarize_calories(prata_image)


Ingredients include:
- Roti Prata, weight: 120g, 30g carbs, 10g protein, 10g fats, 290kcal
- Dhal Curry, weight: 150g, 20g carbs, 10g protein, 5g fats, 145kcal
- Fish Curry, weight: 150g, 15g carbs, 20g protein, 5g fats, 185kcal

Total Calories: 620kcal
Total Carbohydrates = 65g
Total Protein = 40g
Total Fats = 20g

Recommendation for this meal choice:
This meal is high in refined carbohydrates and may be improved by adding a side of vegetables for fiber and nutrients.


## TEST SOME IMAGES

In [29]:
import base64
from mimetypes import guess_type

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

In [30]:
local_image1 = local_image_to_data_url('media/food_image1.jpg')
local_image2 = local_image_to_data_url('media/food_image2.jpg')
local_image3 = local_image_to_data_url('media/food_image3.jpg')


In [31]:
summarize_calories(local_image1)


Ingredients include:
- white rice, weight: 200g, 45g carbs, 4g protein, 1g fats, 197kcal
- sweet and sour fish, weight: 150g, 20g carbs, 30g protein, 5g fats, 245kcal
- mixed vegetables, weight: 100g, 20g carbs, 5g protein, 1g fats, 101kcal
- curry chicken, weight: 180g, 20g carbs, 30g protein, 10g fats, 260kcal

Total Calories: 803kcal
Total Carbohydrates = 105g
Total Protein = 69g
Total Fats = 17g

Recommendation for this meal choice:
This meal has a good variety of protein and vegetables. To make it healthier, consider reducing the portion of white rice to 150g and choose steamed fish over sweet and sour fish to lower the sugar and calorie intake.


In [32]:
summarize_calories(local_image2)


Ingredients include:
- white rice, weight: 200g, 45g carbs, 4g protein, 1g fats, 197kcal
- sweet and sour fish, weight: 150g, 20g carbs, 30g protein, 5g fats, 245kcal
- curry potato, weight: 100g, 20g carbs, 2g protein, 1g fats, 89kcal
- fried chicken, weight: 120g, 10g carbs, 20g protein, 8g fats, 172kcal

Total Calories: 703kcal
Total Carbohydrates = 95g
Total Protein = 56g
Total Fats = 15g

Recommendation for this meal choice:
This meal is high in carbohydrates and fried foods. Consider adding a portion of steamed vegetables to increase fiber intake and balance the meal.


In [33]:
summarize_calories(local_image3)


Ingredients include:
- white rice, weight: 200g, 45g carbs, 4g protein, 1g fats, 197kcal
- stir-fried mixed vegetables, weight: 100g, 15g carbs, 3g protein, 2g fats, 86kcal
- fried egg, weight: 50g, 1g carbs, 6g protein, 5g fats, 73kcal
- black pepper beef, weight: 150g, 10g carbs, 20g protein, 10g fats, 210kcal
- fried tofu, weight: 80g, 10g carbs, 8g protein, 4g fats, 108kcal

Total Calories: 674kcal
Total Carbohydrates = 81g
Total Protein = 41g
Total Fats = 22g

Recommendation for this meal choice:
This meal has a good variety of protein sources and vegetables. However, consider reducing fried items and opting for more steamed or boiled vegetables to lower the overall fat content.
